# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
city_data = pd.read_csv('../output_data/cities.csv')
city_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.6500,115.3333,64.99,76,100,4.00,AU,1614554915
1,1,Rikitea,-23.1203,-134.9692,78.39,77,28,12.19,PF,1614554915
2,2,Ashti,19.3833,76.2333,76.44,25,0,4.70,IN,1614554916
3,3,Atuona,-9.8000,-139.0333,80.37,78,6,16.31,PF,1614554916
4,4,Verkhnyaya Inta,65.9811,60.3094,4.55,92,97,9.75,RU,1614554916
...,...,...,...,...,...,...,...,...,...,...
552,552,Caceres,39.5220,-6.3748,48.99,82,6,3.38,ES,1614555096
553,553,Ozernovskiy,51.5000,156.5167,24.03,94,100,20.15,RU,1614555096
554,554,Vila Velha,-20.3297,-40.2925,82.99,94,40,2.30,BR,1614555080
555,555,Kashgar,39.4547,75.9797,33.80,80,75,2.08,CN,1614555097


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Store 'Lat' and 'Lng' into  locations
locations = city_data[['Lat', 'Lng']].astype(float)

humidity = city_data['Humidity'].astype(float)

In [27]:
# Plot Heatmap
humidity_fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)


# Add layer
humidity_fig.add_layer(heat_layer)

# Display figure
humidity_fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
better_weather = city_data.loc[(city_data['Max Temp'] <= 80) & (city_data['Max Temp'] >= 70)]
better_weather = city_data.loc[(city_data['Cloudiness'] == 0)]

better_weather = better_weather.drop(columns= 'City_ID')
better_weather = better_weather.dropna()
better_weather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
11,castro,0,CL,1558378756,70,-42.48,-73.76,48.20,9.17
13,saskylakh,0,RU,1558378757,91,71.97,114.09,4.92,2.26
26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08
...,...,...,...,...,...,...,...,...,...
527,tomatlan,0,MX,1558378894,59,19.93,-105.25,84.66,7.90
533,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82
538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24
543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = better_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47,
1,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41,
2,castro,0,CL,1558378756,70,-42.48,-73.76,48.20,9.17,
3,saskylakh,0,RU,1558378757,91,71.97,114.09,4.92,2.26,
4,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08,
...,...,...,...,...,...,...,...,...,...,...
68,tomatlan,0,MX,1558378894,59,19.93,-105.25,84.66,7.90,
69,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82,
70,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24,
71,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": gkey
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url

    response = requests.get(base_url, params = params).json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        print(response)

Missing field/result... skipping.
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}


In [8]:
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47,"Shangri-La Hotel, Hohhot"
1,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41,Bangalay Luxury Villas
2,castro,0,CL,1558378756,70,-42.48,-73.76,48.20,9.17,Enjoy Chiloé. Hotel De La Isla
3,saskylakh,0,RU,1558378757,91,71.97,114.09,4.92,2.26,
4,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08,Tirupati Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
humidity_fig.add_layer(markers)

# Display figure
humidity_fig

Figure(layout=FigureLayout(height='420px'))